### **Racer**

##### **Requirements**

In [ ]:
import sys
from typing import Tuple, Union
import pandas as pd
#import torch
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score
sys.path.append("..")
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import time


##### **Preprocess**

In [ ]:
from typing import Tuple, Union

import numpy as np
import pandas as pd

from optbinning import MulticlassOptimalBinning as MOB, MDLP


class RACERPreprocessor:
    def __init__(self, target: str="auto", max_n_bins=32, max_num_splits=32):
        """RACER preprocessing step that quantizes numerical columns and dummy encodes the categorical ones.
        Quantization is based on the optimal binning algorithm for "multiclass" tasks and the entropy-based MDLP
        algorithm for "binary" tasks.

        Args:
            target (str, optional): Whether the task is "multiclass" or "binary" classification. Defaults to "auto" which attempts automatically infer the task from `y`.
            max_n_bins (int, optional): Maximum number of bins to quantize in. Defaults to 32.
            max_num_splits (int, optional): Maximum number of splits to consider at each partition for MDLP. Defaults to 32.
        """
        assert target in ["multiclass", "binary", "auto"], "`target` must either be 'multiclass', 'binary' or 'auto'."
        if target == "multiclass":
            self._quantizer = MOB(max_n_bins=max_n_bins)
        elif target == "binary":
            self._quantizer = MDLP(max_candidates=max_num_splits)
        else:
            self._quantizer = "infer"
            self._max_n_bins = max_n_bins
            self._max_candidates = max_num_splits

    def fit_transform(
        self, X: Union[pd.DataFrame, np.ndarray], y: Union[pd.DataFrame, np.ndarray]
    ) -> Tuple[Union[pd.DataFrame, np.ndarray], Union[pd.DataFrame, np.ndarray]]:
        """Preprocesses the dataset by replacing nominal vaues with dummy variables.
        Converts to numpy boolean arrays and returns the dataset. All numerical values are discretized
        using an optimal binning strategy that employs a decision tree as a preprocessing step.

        Args:
            X (Union[pd.DataFrame, np.ndarray]): Features vector
            y (Union[pd.DataFrame, np.ndarray]): Targets vector

        Returns:
            Tuple[Union[pd.DataFrame, np.ndarray], Union[pd.DataFrame, np.ndarray]]: Transformed features and targets vectors.
        """
        X, y = pd.DataFrame(X), pd.DataFrame(y)
        if self._quantizer == "infer":
            uniques = y.nunique().values
            if uniques > 2:
                self._quantizer = MOB(max_candidates=self._max_num_splits)
            else:
                self._quantizer = MDLP(max_n_bins=self._max_n_bins)
        numerics_X = X.select_dtypes(include=[np.number]).columns.tolist()
        if numerics_X:
            for col in numerics_X:
                self._quantizer.fit(X[col].values, np.squeeze(y.values))
                bins = [X[col].min()] + self._quantizer.splits.tolist() + [X[col].max()]
                X[col] = pd.cut(X[col], bins=bins, include_lowest=True, labels=False)
        X, y = X.astype("category"), y.astype("category")
        X = pd.get_dummies(X).to_numpy()
        y = pd.get_dummies(y).to_numpy()
        return X, y

    def fit(
        self, X: Union[pd.DataFrame, np.ndarray], y: Union[pd.DataFrame, np.ndarray]
    ):
        raise NotImplementedError(
            "Applying transformation across different datasets is not currently supported. Please use fit_transform instead."
        )

    def transform(self):
        raise NotImplementedError(
            "Applying transformation across different datasets is not currently supported. Please use fit_transform instead."
        )

##### **RACER_numpy**


In [ ]:
from typing import Tuple

import numpy as np
from numpy import (
    bitwise_and as AND,
    bitwise_not as NOT,
    bitwise_or as OR,
    bitwise_xor as XOR,
)


def XNOR(input: np.ndarray, other: np.ndarray) -> np.ndarray:
    """Computes the XNOR gate. (semantically the same as `input == other`)

    Args:
        input (np.ndarray): Input array
        other (np.ndarray): Other input array

    Returns:
        np.ndarray: XNOR(input, other) as an array
    """
    return NOT(XOR(input, other))


class RACER:
    def __init__(
        self,
        alpha=0.9,
        suppress_warnings=False,
        benchmark=False,
    ):
        """Initialize the RACER class

        Args:
            alpha (float, optional): Value of alpha according to the RACER paper. Defaults to 0.9.
            suppress_warnings (bool, optional): Whether to suppress any warnings raised during prediction. Defaults to False.
            benchmark (bool, optional): Whether to time the `fit` method for benchmark purposes. Defaults to False.
        """
        self._alpha, self._beta = alpha, 1.0 - alpha
        self._suppress_warnings = suppress_warnings
        self._benchmark = benchmark
        self._has_fit = False

    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        """Fits the RACER algorithm on top of input data X and targets y.
        The code is written in close correlation to the pseudo-code provided in the RACER paper with some slight modifications.

        Args:
            X (np.ndarray): Features vector
            y (np.ndarray): Targets vector
        """
        if self._benchmark:
            from time import perf_counter

            tic = perf_counter()

        self._X, self._y = X, y
        self._cardinality, self._rule_len = self._X.shape
        self._classes = np.unique(self._y, axis=0)
        self._class_indices = {
            self._label_to_int(cls): np.where(XNOR(self._y, cls).min(axis=-1))[0]
            for cls in self._classes
        }

        self._create_init_rules()

        for cls in self._class_indices.keys():
            for i in range(len(self._class_indices[cls])):
                for j in range(i + 1, len(self._class_indices[cls])):
                    self._process_rules(
                        self._class_indices[cls][i], self._class_indices[cls][j]
                    )

        independent_indices = NOT(self._extants_covered)
        self._extants_if, self._extants_then, self._fitnesses = (
            self._extants_if[independent_indices],
            self._extants_then[independent_indices],
            self._fitnesses[independent_indices],
        )

        self._generalize_extants()

        # https://stackoverflow.com/questions/64238462/numpy-descending-stable-arg-sort-of-arrays-of-any-dtype
        args = (
            len(self._fitnesses)
            - 1
            - np.argsort(self._fitnesses[::-1], kind="stable")[::-1]
        )

        self._final_rules_if, self._final_rules_then, self._fitnesses = (
            self._extants_if[args],
            self._extants_then[args],
            self._fitnesses[args],
        )

        self._finalize_rules()

        self._has_fit = True

        if self._benchmark:
            self._bench_time = perf_counter() - tic

    def predict(self, X: np.ndarray, convert_dummies=True) -> np.ndarray:
        """Given input X, predict label using RACER

        Args:
            X (np.ndarray): Input features vector
            convert_dummies (bool, optional): Whether to convert dummy labels back to integert format. Defaults to True.

        Returns:
            np.ndarray: Label as predicted by RACER
        """
        assert self._has_fit, "RACER has not been fit yet."
        labels = np.zeros((len(X), self._final_rules_then.shape[1]), dtype=bool)
        found = np.zeros(len(X), dtype=bool)
        for i in range(len(self._final_rules_if)):
            covered = self._covered(X, self._final_rules_if[i])
            labels[AND(covered, NOT(found))] = self._final_rules_then[i]
            found[covered] = True
            all_found = found.sum() == len(X)
            if all_found:
                break

        if not all_found:
            # if not self._suppress_warnings:
            #     print(
            #         f"WARNING: RACER was unable to find a perfect match for {len(X) - found.sum()} instances out of {len(X)}"
            #     )
            #     print(
            #         "These instances will be labelled as the majority class during training."
            #     )
            leftover_indices = np.where(NOT(found))[0]
            for idx in leftover_indices:
                labels[idx] = self._closest_match(X[idx])

        if convert_dummies:
            labels = np.argmax(labels, axis=-1)

        return labels

    def _bool2str(self, bool_arr: np.ndarray) -> str:
        """Converts a boolean array to a human-readable string

        Args:
            bool_arr (np.ndarray): The input boolean array

        Returns:
            str: Human-readable string output
        """
        return np.array2string(bool_arr.astype(int), separator="")

    def display_rules(self) -> None:
        """Print out the final rules"""
        assert self._has_fit, "RACER has not been fit yet."
        print("Algorithm Parameters:")
        print(f"\t- Alpha: {self._alpha}")
        if self._benchmark:
            print(f"\t- Time to fit: {self._bench_time}s")
        print(
            f"\nFinal Rules ({len(self._final_rules_if)} total): (if --> then (label) | fitness)"
        )
        for i in range(len(self._final_rules_if)):
            print(
                f"\t{self._bool2str(self._final_rules_if[i])} -->"
                f" {self._bool2str(self._final_rules_then[i])}"
                f" ({self._label_to_int(self._final_rules_then[i])})"
                f" | {self._fitnesses[i]}"
            )

    def _closest_match(self, X: np.ndarray) -> np.ndarray:
        """Find the closest matching rule to `X` (This will be extended later)

        Args:
            X (np.ndarray): Input rule `X`

        Returns:
            np.ndarray: Matched rule
        """
        return self._majority_then

    def score(self, X_test: np.ndarray, y_test: np.ndarray) -> float:
        """Returns accuracy on the provided test data.

        Args:
            X_test (np.ndarray): Test features vector
            y_test (np.ndarray): Test targets vector

        Returns:
            float: Accuracy score
        """
        assert self._has_fit, "RACER has not been fit yet."
        try:
            from sklearn.metrics import accuracy_score
        except ImportError as e:
            raise ImportError(
                "scikit-learn is required to use the score function. Install wit `pip install scikit-learn`."
            )
        if y_test.ndim != 1 and y_test.shape[1] != 1:
            y_test = np.argmax(y_test, axis=-1)
        y_pred = self.predict(X_test)
        return accuracy_score(y_test, y_pred)

    def _fitness_fn(self, rule_if: np.ndarray, rule_then: np.ndarray) -> np.ndarray:
        """Returns fitness for a given rule according to the RACER paper

        Args:
            rule_if (np.ndarray): If part of a rule (x)
            rule_then (np.ndarray): Then part of a rule (y)

        Returns:
            np.ndarray: Fitness score for the rule as defined in the RACER paper
        """
        n_covered, n_correct = self._confusion(rule_if, rule_then)
        accuracy = n_correct / n_covered
        coverage = n_covered / self._cardinality
        return self._alpha * accuracy + self._beta * coverage

    def _covered(self, X: np.ndarray, rule_if: np.ndarray) -> np.ndarray:
        """Returns indices of instances if `X` that are covered by `rule_if`.
        Note that rule covers instance if EITHER of the following holds in a bitwise manner:
        1. instance[i] == 0
        2. instance[i] == 1 AND rule[i] == 1

        Args:
            X (np.ndarray): Instances
            rule_if (np.ndarray): If part of rule (x)

        Returns:
            np.ndarray: An array containing indices in `X` that are covered by `rule_if`
        """
        covered = OR(NOT(X), AND(rule_if, X)).min(axis=-1)
        return covered

    def _confusion(
        self, rule_if: np.ndarray, rule_then: np.ndarray
    ) -> Tuple[np.ndarray, np.ndarray]:
        """Returns n_correct and n_covered for instances classified by a rule.

        Args:
            rule_if (np.ndarray): If part of rule (x)
            rule_then (np.ndarray): Then part of rule (y)

        Returns:
            Tuple[np.ndarray, np.ndarray]: (n_covered, n_correct)
        """
        covered = self._covered(self._X, rule_if)
        n_covered = covered.sum()
        y_covered = self._y[covered]
        n_correct = XNOR(y_covered, rule_then).min(axis=-1).sum()
        return n_covered, n_correct

    def _get_majority(self) -> np.ndarray:
        """Return the majority rule_then from self._y

        Returns:
            np.ndarray: Majority rule_then
        """
        u, indices = np.unique(self._y, axis=0, return_inverse=True)
        return u[np.bincount(indices).argmax()]

    def _create_init_rules(self) -> None:
        """Creates an initial set of rules from theinput feature vectors"""
        self._extants_if = self._X.copy()
        self._extants_then = self._y.copy()
        self._extants_covered = np.zeros(len(self._X), dtype=bool)
        self._majority_then = self._get_majority()
        self._fitnesses = np.array(
            [
                self._fitness_fn(rule_if, rule_then)
                for rule_if, rule_then in zip(self._X, self._y)
            ]
        )

    def _composable(self, idx1: int, idx2: int) -> bool:
        """Returns true if two rules indicated by their indices are composable

        Args:
            idx1 (int): Index of the first rule
            idx2 (int): Index of the second rule

        Returns:
            bool: True if labels match and neither of the rules are covered. False otherwise.
        """
        labels_match = XNOR(self._extants_then[idx1], self._extants_then[idx2]).min()
        return (
            labels_match
            and not self._extants_covered[idx1]
            and not self._extants_covered[idx2]
        )

    def _process_rules(self, idx1: int, idx2: int) -> None:
        """Process two rules indiciated by their indices

        Args:
            idx1 (int): Index of the first rule
            idx2 (int): Index of the second rule
        """
        if self._composable(idx1, idx2):
            composition = self._compose(self._extants_if[idx1], self._extants_if[idx2])
            composition_fitness = self._fitness_fn(
                composition, self._extants_then[idx1]
            )
            if composition_fitness > np.maximum(
                self._fitnesses[idx1], self._fitnesses[idx2]
            ):
                self._update_extants(
                    idx1, composition, self._extants_then[idx1], composition_fitness
                )

    def _compose(self, rule1: np.ndarray, rule2: np.ndarray) -> np.ndarray:
        """Composes rule1 with rule2

        Args:
            rule1 (np.ndarray): The first rule
            rule2 (np.ndarray): The second rule

        Returns:
            np.ndarray: The composed rule which is simply the bitwise OR of the two rules
        """
        return OR(rule1, rule2)

    def _update_extants(
        self,
        index: int,
        new_rule_if: np.ndarray,
        new_rule_then: np.ndarray,
        new_rule_fitness: np.ndarray,
    ):
        """Remove all rules from current extants that are covered by `new_rule`.
        Then append new rule to extants.

        Args:
            index (int): Index of `new_rule`
            new_rule_if (np.ndarray): If part of `new_rule` (x)
            new_rule_then (np.ndarray): Then part of `new_rule` (y)
            new_rule_fitness (np.ndarray): Fitness of the `new_rule`
        """
        same_class_indices = self._class_indices[self._label_to_int(new_rule_then)]
        covered = self._covered(self._extants_if[same_class_indices], new_rule_if)
        self._extants_covered[same_class_indices[covered]] = True
        self._extants_covered[index] = False
        self._extants_if[index], self._extants_then[index], self._fitnesses[index] = (
            new_rule_if,
            new_rule_then,
            new_rule_fitness,
        )

    def _label_to_int(self, label: np.ndarray) -> int:
        """Converts dummy label to int

        Args:
            label (np.ndarray): Label to convert

        Returns:
            int: Converted label
        """
        return int(np.argmax(label))

    def _generalize_extants(self) -> None:
        """Generalize the extants by flipping every 0 to a 1 and checking if the fitness improves."""
        new_extants_if = np.zeros_like(self._extants_if, dtype=bool)
        for i in range(len(self._extants_if)):
            for j in range(len(self._extants_if[i])):
                if not self._extants_if[i][j]:
                    self._extants_if[i][j] = True
                    fitness = self._fitness_fn(
                        self._extants_if[i], self._extants_then[i]
                    )
                    if fitness > self._fitnesses[i]:
                        self._fitnesses[i] = fitness
                    else:
                        self._extants_if[i][j] = False
            new_extants_if[i] = self._extants_if[i]
        self._extants_if = new_extants_if

    def _finalize_rules(self) -> None:
        """Removes redundant rules to form the final ruleset"""
        temp_rules_if = self._final_rules_if
        temp_rules_then = self._final_rules_then
        temp_rules_fitnesses = self._fitnesses
        i = 0
        while i < len(temp_rules_if) - 1:
            mask = np.ones(len(temp_rules_if), dtype=bool)
            covered = self._covered(temp_rules_if[i + 1 :], temp_rules_if[i])
            mask[i + 1 :][covered] = False
            temp_rules_if, temp_rules_then, temp_rules_fitnesses = (
                temp_rules_if[mask],
                temp_rules_then[mask],
                temp_rules_fitnesses[mask],
            )
            i += 1

        self._final_rules_if, self._final_rules_then, self._fitnesses = (
            temp_rules_if,
            temp_rules_then,
            temp_rules_fitnesses,
        )
        
# *********************************************************************************************************************************
    
    def raceknn(self, X: np.ndarray, X_train: np.ndarray, Y_train: np.ndarray, kmeasure: int, convert_dummies=True) -> np.ndarray:
        """Given input X, predict label using RACER

    Args:
        X (np.ndarray): input features vector
        X_train (np.ndarray): training features vector
        Y_train (np.ndarray): training labels vector
        kmeasure (int): the minimum number of samples required to assign a label using majority voting
        convert_dummies (bool): whether to convert the output to a one-dimensional array

    Returns:
        np.ndarray: label as predicted by RACER
    """
        assert self._has_fit, "RACER has not been fit yet."
        labelss = np.zeros((len(X), self._final_rules_then.shape[1]), dtype=bool)
        founds = np.zeros(len(X), dtype=bool)
        xy__train = [None] * len(X)
        for i in range(len(self._final_rules_if)):
            covereds = self._covered(X, self._final_rules_if[i])
            xtrain_cover = self._covered(X_train, self._final_rules_if[i])
            if i < len(X) :
                bes_rulei = self._closest_match_if(X[i])
            for indi,vali in enumerate(bes_rulei) :
                if xtrain_cover.sum() < kmeasure :
                    covi = self._covered(X_train, bes_rulei[indi])
                    xtrain_cover = OR(xtrain_cover, covi)
                        
            labelss[AND(covereds, NOT(founds))] = self._final_rules_then[i]
            founds[covereds] = True
            for index,val in enumerate(covereds):
                if val:
                    xy__train[index] = xtrain_cover
            if founds.sum() == len(X):  # -> every instance was matched to a rule
                break

        all_found = founds.sum() == len(X)
        if not all_found:
            # print(
            #     f"Warning: RACER was unable to find a perfect match for {len(X) - founds.sum()} instances out of {len(X)}."
            # )
            # print(
            #     "Labels for these instances will be determined by a closest match algorithm."
            # )
            leftover_indices = np.where(~founds)[0]
            #print(leftover_indices)
            for idx in leftover_indices:
                labelss[idx] = self._closest_match(X[idx])
                bes_rule = self._closest_match_if(X[idx])
                xtrain_closest = self._covered(X_train, bes_rule[0])
                for indl,vall in enumerate(bes_rule) :
                    if xtrain_closest.sum() < kmeasure :
                        covl = self._covered(X_train, bes_rule[indl])
                        xtrain_closest = OR(xtrain_closest, covl)
                        
                    xy__train[idx] = xtrain_closest
                    
        if convert_dummies:
            labelss = np.argmax(labelss, axis=-1)

        return xy__train

    def _closest_match_if(self, X: np.ndarray) -> np.ndarray:
        """Find the closest matching rule to `X`
        Args:
            X (np.ndarray): input `X`
        Returns:
            np.ndarray: matched rule
        """
        # coverage := count of covered bits by a rule. Higher is better.
        gum = 0.6
        int_X = X.astype(int)  # <- cast boolean array to integer array
        overlap = OR(NOT(int_X), AND(self._final_rules_if, int_X)).sum(axis=-1)
        overlap = overlap / self._rule_len  # -> normalize by rule length
        scores = np.multiply(gum * overlap, (1 - gum) * self._fitnesses)
        scores = np.argsort(-scores)
        final_best_rules = self._final_rules_if[scores]
        return final_best_rules
    
    def _closest_match_then(self, X: np.ndarray) -> np.ndarray:
            """Find the closest matching rule to `X`
            Args:
                X (np.ndarray): input `X`
            Returns:
                np.ndarray: matched rule
            """
            # coverage := count of covered bits by a rule. Higher is better.
            gum = 0.6
            int_X = X.astype(int)  # <- cast boolean array to integer array
            overlap = OR(NOT(int_X), AND(self._final_rules_if, int_X)).sum(axis=-1)
            overlap = overlap / self._rule_len  # -> normalize by rule length
            scores = np.multiply(gum * overlap, (1 - gum) * self._fitnesses)
            scores = np.argsort(-scores)
            final_best_then = self._final_rules_then[scores]
            return final_best_then

### **Dataset &  Results**

In [ ]:
# Set initial values
measure_k = 3
Alpha = 0.9

##### **Load Dataset and Split that**

In [ ]:
df = pd.read_csv(
    "",
    names=[]
)

print("Dataset information:\n", df.info())

print("\nDataset description:\n", df.describe())


# Spiliting X & Y in dataset
X = df.drop(columns=['class']).astype('category')
Y = df[['class']].astype('category')
print("\nThe X datset:\n", X.head(), "\n\nThe Y dataset:\n", Y.head())

# Fitting RACERPreprocessor() on dataset
X, Y = RACERPreprocessor(target="multiclass").fit_transform(X, Y)

# Defining the KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

##### **RACEkNN**

In [ ]:
KNNETimePerFold = []
FinalAccuracyKNNEPerfold = []

# Edited KNN
KNNE = KNeighborsClassifier(n_neighbors=measure_k)

print("K measure is:", measure_k, "\tAlpha measure is:", Alpha)

# Initialize Racer with alpha measure
racer = RACER(alpha=Alpha, suppress_warnings=False, benchmark=True)

# Define lists to store execution time and accuracy per fold
RacerTimePerFold = []
FinalAccuracyRacerPerFold = []

# Perform cross-validation
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    
    RacerST = time.time()  # Start timer
    racer.fit(X_train, Y_train)
    prediction = racer.score(X_test, Y_test)  # Predict for a single test data point
    RacerET = time.time()  # End timer
        
    KNNEESDifference = []
    PredictionsKNNEPerFold = []  # Clear predictions list for each fold
    
    for i in range(len(X_test)):
        xy = racer.raceknn(X_test[i:i+1], X_train, Y_train, measure_k, True)
        x__train = [X_train[v] for v in xy]
        y__train = [Y_train[v] for v in xy]
        KNNEST = time.time()  # Start timer
        KNNE.fit(x__train[i], y__train[i])
        predictionE = KNNE.predict([X_test[i]])  # Predict for a single test data point
        KNNEET = time.time()  # End timer
        
        PredictionsKNNEPerFold.append(predictionE[0])  # Append the predicted label (assumes single label prediction)
        KNNEESDifference.append(KNNEET - KNNEST)  # Calculate elapsed time
    
    KNNETimePerFold.append(sum(KNNEESDifference)) 
    FinalAccuracyKNNEPerfold.append(accuracy_score(Y_test, PredictionsKNNEPerFold))  # Compute accuracy for each fold

print("\nAverage execution time across all folds (EKNN):", np.mean(KNNETimePerFold))
print("\nAverage accuracy across all folds (EKNN):", np.mean(FinalAccuracyKNNEPerfold))